In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

In [3]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# Load zero-shot DistilGPT2 pipeline
generator = pipeline("text-generation", model="distilgpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
# Load your train and test data
train_df = pd.read_excel("/content/drive/My Drive/Colab Notebooks/DATASCI 266/Final Project/train_set.xlsx")
test_df = pd.read_excel("/content/drive/My Drive/Colab Notebooks/DATASCI 266/Final Project/test_set.xlsx")

In [5]:
print(test_df[['line1', 'line2']].head())

                                               line1  \
0                      And put my head on his  bible   
1  Of the picture—punchline, figured out, "Ahh, I...   
2        Got the world followin' the New York script   
3  Tell it like it is nigga, tell it like it is h...   
4                     Yeah, think about me, baby, oh   

                                               line2  
0                      I swear to God I didn't do it  
1  No, you don't, nigga, so why don't you go'n' f...  
2  Hustle with Timbs and hoodies on my New York flip  
3  Tell it like it is nigga, tell it like it is, ...  
4                           Think about me, baby, oh  


In [6]:
# Define dynamic prompt function
def build_prompt(line1):
    return f"{str(line1).strip()} \nNext line:"

In [7]:
# Use a smaller sample for baseline (e.g., 500 rows)
sample_df = test_df.head(500)

# Add truncation and limit new tokens only
test_outputs = []

for _, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    line1 = row["line1"]
    expected_line2 = row["line2"]

    prompt = build_prompt(line1)
    try:
        output = generator(
            prompt,
            max_new_tokens=30,             # More efficient
            num_return_sequences=1,
            truncation=True,               # Avoids warning
            pad_token_id=50256             # EOS token for GPT2/DistilGPT2
        )
        # Only take newly generated part, clean output
        generated_line2 = output[0]["generated_text"].replace(prompt, "").strip()
    except Exception as e:
        generated_line2 = f"ERROR: {e}"

    test_outputs.append({
        "line1": line1,
        "expected_line2": expected_line2,
        "generated_line2": generated_line2,
        "prompt_used": prompt
    })


100%|██████████| 500/500 [01:59<00:00,  4.19it/s]


In [8]:
results_df = pd.DataFrame(test_outputs)
results_df.sample(5)

,line1,expected_line2,generated_line2,prompt_used
374,I was rich when I was pissin',"On myself, I'm ballin'",I'm a kid who is so fucking tired of being a k...,I was rich when I was pissin' \nNext line:
490,She was suckin' my dick behind her old man bac...,"She like to gangbang, like the way I throw it ...","- I should be paying attention to this man, hi...",She was suckin' my dick behind her old man bac...
246,"Take a little break to hit me, hit me back",Go unless you make it now,-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-,"Take a little break to hit me, hit me back \nN..."
2,Got the world followin' the New York script,Hustle with Timbs and hoodies on my New York flip,You've seen the movie now.\nNow you're gonna b...,Got the world followin' the New York script \n...
358,Then go down on each other (Yeah),I put the vibe under covers (Yeah),Then go to your place and go to your place and...,Then go down on each other (Yeah) \nNext line:


In [9]:
!pip install transformers datasets


In [10]:
import pandas as pd
import random

# Load training data
train_df = pd.read_excel("/content/drive/My Drive/Colab Notebooks/DATASCI 266/Final Project/train_set.xlsx")
train_df = train_df.dropna(subset=["line1", "line2"])

# Dummy function — replace with actual emotion detection if available
def get_emotion(text):
    # placeholder — use Spotify tag or emotion classifier
    return random.choice(["joy", "sadness", "anger", "love", "fear", "surprise"])

def mask_random_word(text):
    words = text.split()
    if len(words) < 2:
        return text, None
    idx = random.randint(0, len(words) - 1)
    masked = words[idx]
    words[idx] = "<mask>"
    return " ".join(words), masked

inputs = []

for _, row in train_df.iterrows():
    line1 = row["line1"]
    line2 = row["line2"]

    masked_line2, masked_word = mask_random_word(line2)
    if not masked_word:
        continue

    emotion = get_emotion(line2)
    prompt = f"Line 1: {line1.strip()}\nLine 2: {masked_line2.strip()} (Emotion: {emotion})"
    inputs.append(prompt)

In [12]:
from datasets import Dataset

dataset = Dataset.from_dict({"text": inputs})
dataset = dataset.train_test_split(test_size=0.1)

# Tokenization
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # required for GPT2 padding

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/70842 [00:00<?, ? examples/s]

Map:   0%|          | 0/7872 [00:00<?, ? examples/s]

In [14]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

model = GPT2LMHeadModel.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-lyric-mask",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    save_steps=500,
    eval_strategy="epoch",
    fp16=True,
    max_steps=5000,
    report_to="wandb",  # Optional
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipython-input-14-3725840492.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
0,2.493400,2.564854


TrainOutput(global_step=5000, training_loss=2.6207720001220705, metrics={'train_runtime': 794.7489, 'train_samples_per_second': 25.165, 'train_steps_per_second': 6.291, 'total_flos': 1306460160000000.0, 'train_loss': 2.6207720001220705, 'epoch': 0.28231042854723054})

In [17]:
trainer.save_model("./gpt2-lyric-mask")  # Save fine-tuned model
tokenizer.save_pretrained("./gpt2-lyric-mask")  # Save tokenizer for future use

('./gpt2-lyric-mask/tokenizer_config.json',
 './gpt2-lyric-mask/special_tokens_map.json',
 './gpt2-lyric-mask/vocab.json',
 './gpt2-lyric-mask/merges.txt',
 './gpt2-lyric-mask/added_tokens.json')

In [18]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the fine-tuned model
model_path = "./gpt2-lyric-mask"  # or your full output_dir
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token  # for safety

model.eval()

# Example prompt (same format as training)
input_prompt = "Line 1: I’m lost in a world of dreams\nLine 2: My heart <mask> in the night"

# Tokenize and convert to input tensor
inputs = tokenizer(input_prompt, return_tensors="pt")

# Generate prediction
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        max_length=50,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode and print
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Output:\n", generated)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Output:
 Line 1: I’m lost in a world of dreams
Line 2: My heart <mask> in the night (Emotion: joy) (Emotion: surprise) (Emotion: anger) (Emotion 1: sadness)


In [19]:
prompt = "Line 1: I’m lost in a world of dreams\nLine 2: My heart"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Output:\n", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Output:
 Line 1: I’m lost in a world of dreams
Line 2: My heart is <mask> (Emotion: sadness) (Yeah) I'm so sorry (I'm sorry), I don't know what to do now (
